In [1]:
import pyspark
import dxpy
import dxdata
import toml
from pathlib import Path

import hail as hl

In [2]:
sc = pyspark.SparkContext()
spark = pyspark.sql.SparkSession(sc)

In [3]:
with open("../config.toml") as f:
    conf = toml.load(f)

log_file = Path(conf["IMPORT"]["LOG_DIR"], f"annotate.log").resolve().__str__()
GENE = "GCGR"

In [4]:
hl.init(sc=sc, default_reference='GRCh38', log=log_file)

pip-installed Hail requires additional configuration options in Spark referring
  to the path to the Hail Python module directory HAIL_DIR,
  e.g. /path/to/python/site-packages/hail:
    spark.jars=HAIL_DIR/hail-all-spark.jar
    spark.driver.extraClassPath=HAIL_DIR/hail-all-spark.jar
    spark.executor.extraClassPath=./hail-all-spark.jarRunning on Apache Spark version 2.4.4
SparkUI available at http://ip-10-60-85-84.eu-west-2.compute.internal:8081
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.61-3c86d3ba497a
LOGGING: writing to /opt/notebooks/gogoGPCR/hail_logs/annotate.log


In [5]:
stage = "final"
#write_file = "file:" + "/mnt/project/data/matrix_tables/GCGR.final.mt"
write_file = "file:/opt/notebooks/gogoGPCR/tmp/GCGR.final.mt"

mt = hl.read_matrix_table(write_file)
mt.count()


(297, 162483)

In [6]:
cadd_file = "file:" + "/mnt/project/data/GCGR_CADD.tsv.bgz"

ht = hl.import_table(
        cadd_file,
        impute=True,
        comment="#",
        find_replace=("^#", ""),
    )

ht.count()

2021-09-15 15:44:25 Hail: INFO: Reading table to impute column types
2021-09-15 15:44:30 Hail: INFO: Loading 134 fields. Counts by type:
  float64: 60
  int32: 52
  str: 22


41265

In [7]:
ht = ht.annotate(Chrom = "chr" + hl.str(ht.Chrom))

ht = ht.annotate(
        locus=hl.locus(ht.Chrom, ht.Pos, reference_genome="GRCh38"),
        alleles=hl.array([ht.Ref, ht.Alt]),
    )

ht = ht.key_by(ht.locus, ht.alleles)

ht = ht.filter((ht.ConsDetail == "missense"))

mt = mt.annotate_rows(cadd=ht[mt.locus, mt.alleles])

In [8]:
mt = mt.filter_rows(hl.is_defined(mt.cadd))

In [9]:
mt.count()

2021-09-15 15:44:33 Hail: INFO: Coerced sorted dataset


(170, 162483)

In [10]:
anno = hl.import_table("file:/mnt/project/data/annotations/GCGR_LoF.txt", key = "Variant")

2021-09-15 15:44:35 Hail: INFO: Reading table without type imputation
  Loading field 'Variant' as type str (not specified)
  Loading field 'LoF' as type str (not specified)


In [11]:
mt = mt.annotate_rows(protCons = mt.cadd.oAA + hl.str(mt.cadd.protPos) + mt.cadd.nAA)

In [12]:
mt = mt.annotate_rows(LoF = anno[mt.protCons].LoF)

In [13]:
mt = hl.variant_qc(mt)

In [32]:
mt2 = mt.filter_rows(mt.LoF == "LoF")
mt2.select_rows(mt2.variant_qc, mt2.protCons, mt2.LoF).rows().show(-1)

2021-09-15 15:59:33 Hail: INFO: Coerced sorted dataset
2021-09-15 15:59:34 Hail: INFO: Coerced sorted dataset
2021-09-15 15:59:36 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-09-15 15:59:36 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-09-15 15:59:38 Hail: INFO: Ordering unsorted dataset with network shuffle


+----------------+------------+--------------------------+
| locus          | alleles    | variant_qc.dp_stats.mean |
+----------------+------------+--------------------------+
| locus<GRCh38>  | array<str> |                  float64 |
+----------------+------------+--------------------------+
| chr17:81810848 | ["G","A"]  |                 3.88e+01 |
| chr17:81810923 | ["C","A"]  |                 3.90e+01 |
| chr17:81811111 | ["G","T"]  |                 6.33e+01 |
| chr17:81811122 | ["T","G"]  |                 6.33e+01 |
| chr17:81811667 | ["G","A"]  |                 7.00e+01 |
| chr17:81812226 | ["C","T"]  |                 1.93e+01 |
| chr17:81812853 | ["G","A"]  |                 4.12e+01 |
| chr17:81812871 | ["G","A"]  |                 2.60e+01 |
+----------------+------------+--------------------------+

+---------------------------+-------------------------+
| variant_qc.dp_stats.stdev | variant_qc.dp_stats.min |
+---------------------------+-------------------------+
|                   float64 |                 float64 |
+---------------------------+-------------------------+
|                  1.07e+01 |                1.10e+01 |
|                  1.08e+01 |                1.60e+01 |
|                  1.32e+01 |                1.60e+01 |
|                  1.32e+01 |                1.60e+01 |
|                  1.69e+01 |                1.00e+01 |
|                  3.75e+00 |                1.60e+01 |
|                  2.56e+01 |                1.60e+01 |
|                  1.13e+01 |                1.60e+01 |
+---------------------------+-------------------------+

+-------------------------+--------------------------+
| variant_qc.dp_stats.max | variant_qc.gq_stats.mean |
+-------------------------+--------------------------+
|                 float64 |                  float64 |
+-------------------------+--------------------------+
|                8.60e+01 |                 4.97e+01 |
|                1.04e+02 |                 4.97e+01 |
|                1.27e+02 |                 5.00e+01 |
|                1.49e+02 |                 5.00e+01 |
|                1.65e+02 |                 5.00e+01 |
|                8.90e+01 |                 4.91e+01 |
|                2.15e+02 |                 4.98e+01 |
|                2.09e+02 |                 4.97e+01 |
+-------------------------+--------------------------+

+---------------------------+-------------------------+
| variant_qc.gq_stats.stdev | variant_qc.gq_stats.min |
+---------------------------+-------------------------+
|                   float64 |                 float64 |
+---------------------------+-------------------------+
|                  6.71e-01 |                3.00e+01 |
|                  7.85e-01 |                2.10e+01 |
|                  2.15e-01 |                3.10e+01 |
|                  2.16e-01 |                3.60e+01 |
|                  1.65e-01 |                3.00e+01 |
|                  9.99e-01 |                3.30e+01 |
|                  6.56e-01 |                3.20e+01 |
|                  7.41e-01 |                4.00e+01 |
+---------------------------+-------------------------+

+-------------------------+---------------+---------------------+
| variant_qc.gq_stats.max | variant_qc.AC | variant_qc.AF       |
+-------------------------+---------------+---------------------+
|                 float64 | array<int32>  | array<float64>      |
+-------------------------+---------------+---------------------+
|                5.00e+01 | [324964,2]    | [1.00e+00,6.15e-06] |
|                5.00e+01 | [324922,30]   | [1.00e+00,9.23e-05] |
|                5.50e+01 | [324965,1]    | [1.00e+00,3.08e-06] |
|                5.50e+01 | [324963,3]    | [1.00e+00,9.23e-06] |
|                5.70e+01 | [324964,2]    | [1.00e+00,6.15e-06] |
|                5.00e+01 | [324964,2]    | [1.00e+00,6.15e-06] |
|                5.00e+01 | [324948,18]   | [1.00e+00,5.54e-05] |
|                5.10e+01 | [324917,49] 

In [15]:
def add_varid(mt: hl.matrixtable.MatrixTable) -> hl.matrixtable.MatrixTable:

    mt = mt.annotate_rows(
        varid=hl.delimit(
            [
                mt.locus.contig,
                hl.str(mt.locus.position),
                mt.alleles[0],
                mt.alleles[1],
            ],
            ":",
        )
    )

    return mt


def recode_GT_to_GP(
    mt: hl.matrixtable.MatrixTable,
) -> hl.matrixtable.MatrixTable:

    GPs = hl.literal([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]])

    mt = mt.annotate_entries(GP=GPs[mt.GT.n_alt_alleles()])

    return mt


def write_bgen(mt: hl.matrixtable.MatrixTable, output: str) -> None:

    mt = add_varid(mt)

    mt = recode_GT_to_GP(mt)

    hl.export_bgen(
        mt=mt, varid=mt.varid, rsid=mt.varid, gp=mt.GP, output=output
    )

In [17]:
write_bgen(mt, "file:/opt/notebooks/gogoGPCR/tmp/GCGR")

2021-09-15 15:48:51 Hail: INFO: Coerced sorted dataset
2021-09-15 15:49:08 Hail: INFO: while writing:
    file:/opt/notebooks/gogoGPCR/tmp/GCGR.bgen
  merge time: 32.451ms


In [19]:
mt = mt.annotate_rows(LoF2 = hl.if_else(mt.LoF == "LoF", "LoF", "WT", missing_false = True))

In [25]:
mt = add_varid(mt)

annotations = (
    mt.select_rows(
        mt.varid,
        mt.cadd.GeneName,
        mt.LoF2
    )
    .rows()
    .key_by("varid")
    .drop("locus")
    .drop("alleles")
)
annotations.export("file:/opt/notebooks/gogoGPCR/tmp/GCGR.annotations", header=False)

2021-09-15 15:56:14 Hail: INFO: Coerced sorted dataset
2021-09-15 15:56:15 Hail: INFO: Coerced sorted dataset
2021-09-15 15:56:17 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-09-15 15:56:17 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-09-15 15:56:19 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-09-15 15:56:20 Hail: INFO: Coerced sorted dataset
2021-09-15 15:56:22 Hail: INFO: merging 2 files totalling 4.5K...
2021-09-15 15:56:22 Hail: INFO: while writing:
    file:/opt/notebooks/gogoGPCR/tmp/GCGR.annotations
  merge time: 25.268ms


In [29]:
position = mt.aggregate_rows(hl.agg.min(mt.locus.position))
names = mt.varid.collect()
names_str = ",".join(names)

line = f"{mt.cadd.GeneName.collect()[0]}\t{mt.locus.contig.collect()[0]}\t{position}\t{names_str}"

with open(f"/opt/notebooks/gogoGPCR/tmp/GCG.setlist", "w") as f:
    f.write(line)

2021-09-15 15:58:10 Hail: INFO: Coerced sorted dataset
2021-09-15 15:58:14 Hail: INFO: Coerced sorted dataset
2021-09-15 15:58:17 Hail: INFO: Coerced sorted dataset
2021-09-15 15:58:20 Hail: INFO: Coerced sorted dataset
